# Optimize Marketing Campaigns

This tutorial follows the instruction of the [video](https://youtu.be/GHxOtLtZqWU?si=QC-e4CjFUe_teaeq) from Abset Data.

# Introduction

- What is the impact of each marketing strategy and sales visit on Sales (Amount Collected)?
- Is the same strategy valid for all the different client types?

# Data loading and quality

In [307]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
###
import os
from io import StringIO

In [308]:
dir_proj = os.getcwd()
DF = pd.read_csv(dir_proj+"\Campaign-Data.csv", header=0)

In [309]:
DF.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
0,ID-987275,Medium Facility,2800,125,1003,16-01-2014,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
1,ID-987275,Medium Facility,2800,125,1003,16-02-2014,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low
2,ID-987275,Medium Facility,2800,125,1003,18-03-2014,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
3,ID-987275,Medium Facility,2800,125,1003,18-04-2014,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low
4,ID-987275,Medium Facility,2800,125,1003,19-05-2014,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low


In [310]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Client ID              2976 non-null   object 
 1   Client Type            2976 non-null   object 
 2   Number of Customers    2976 non-null   int64  
 3   Montly Target          2976 non-null   int64  
 4   Zip Code               2976 non-null   int64  
 5   Calendardate           2976 non-null   object 
 6   Amount Collected       2976 non-null   int64  
 7   Unit Sold              2976 non-null   int64  
 8   Campaign (Email)       2976 non-null   float64
 9   Campaign (Flyer)       2976 non-null   float64
 10  Campaign (Phone)       2976 non-null   float64
 11  Sales Contact 1        2976 non-null   float64
 12  Sales Contact 2        2976 non-null   float64
 13  Sales Contact 3        2976 non-null   float64
 14  Sales Contact 4        2976 non-null   float64
 15  Sale

In [311]:
DF.describe(include='all')

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
count,2976,2976,2976.000000,2976.000000,2976.0,2976,2.976000e+03,2976.000000,2.976000e+03,2.976000e+03,2.976000e+03,2.976000e+03,2.976000e+03,2.976000e+03,2.976000e+03,2.976000e+03,2976
unique,124,4,NaN,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,ID-987275,Large Facility,NaN,NaN,NaN,16-01-2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low
freq,24,1368,NaN,NaN,NaN,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2480
mean,NaN,NaN,1456.935484,75.080645,1003.0,NaN,1.700440e+07,121.464718,1.432850e+05,6.854186e+05,2.977743e+04,1.313407e+05,2.023319e+06,1.867487e+06,7.076361e+04,1.462954e+04,NaN
std,NaN,NaN,1669.848734,87.042114,0.0,NaN,3.025803e+07,216.413984,7.230452e+05,1.727587e+06,3.832134e+05,1.034882e+06,3.396991e+06,3.271349e+06,3.869872e+05,8.905955e+04,NaN
min,NaN,NaN,0.000000,5.000000,1003.0,NaN,-2.216150e+07,-63.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25%,NaN,NaN,240.000000,10.000000,1003.0,NaN,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
50%,NaN,NaN,960.000000,47.500000,1003.0,NaN,3.409460e+06,24.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.450000e+05,3.225000e+05,0.000000e+00,0.000000e+00,NaN
75%,NaN,NaN,2090.000000,101.250000,1003.0,NaN,2.045676e+07,147.000000,0.000000e+00,8.148285e+04,0.000000e+00,0.000000e+00,2.580000e+06,2.580000e+06,0.000000e+00,0.000000e+00,NaN


# Feature additions and engineering

In [313]:
DF['Calendardate'].unique()

array(['16-01-2014', '16-02-2014', '18-03-2014', '18-04-2014',
       '19-05-2014', '16-06-2014', '17-07-2014', '16-08-2014',
       '16-09-2014', '16-10-2014', '16-11-2014', '17-12-2014',
       '16-01-2015', '16-02-2015', '18-03-2015', '18-04-2015',
       '19-05-2015', '16-06-2015', '17-07-2015', '16-08-2015',
       '16-09-2015', '16-10-2015', '16-11-2015', '17-12-2015'],
      dtype=object)

In [314]:
DF['Calendardate'] = pd.to_datetime(DF['Calendardate'], format='%d-%m-%Y')
DF['Calendar_Month'] = DF['Calendardate'].dt.month
DF['Calendar_year'] = DF['Calendardate'].dt.year

# Exploratory data analysis and statistical analysis

We can have a detailed exploration that can be added to this section, but since we only need to answer three questions:

1. Exploring and understanding basics data
    1. Distribution of data across different accounts
    2. Difference of sales in account types (using categorical mean)
2. Statistical analysis - answering the questions
    1. Impact of markering strategy on sales (using correlation, regression and decision tree)
    2. Impact of competition sales
    3. How different types of client can have different strategies (broken down question 1 and question 2 based on account type)

## Impact of marketing strategy on sales

In [318]:
DF['Client Type'].value_counts(normalize=True)

Client Type
Large Facility      0.459677
Small Facility      0.282258
Medium Facility     0.169355
Private Facility    0.088710
Name: proportion, dtype: float64

In [319]:
pd.crosstab(DF['Number of Competition'], DF['Client Type'], margins=True, normalize=True)

Client Type,Large Facility,Medium Facility,Private Facility,Small Facility,All
Number of Competition,,,,,
High,0.076613,0.028226,0.014785,0.047043,0.166667
Low,0.383065,0.141129,0.073925,0.235215,0.833333
All,0.459677,0.169355,0.088710,0.282258,1.000000


In [320]:
DF.groupby('Number of Competition').mean(numeric_only=True)

,Number of Customers,Montly Target,Zip Code,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Calendar_Month,Calendar_year
Number of Competition,,,,,,,,,,,,,,,
High,1456.935484,75.080645,1003.0,2.974789e+07,213.127016,105398.938508,994046.717540,45198.036895,146945.564516,2.685333e+06,1.786754e+06,72172.379032,8452.620968,10.5,2015.0
Low,1456.935484,75.080645,1003.0,1.445570e+07,103.132258,150862.165766,623692.979839,26693.304194,128219.758065,1.890916e+06,1.883634e+06,70481.854839,15864.919355,5.7,2014.4


In [321]:
DF.groupby('Client Type').mean(numeric_only=True)

,Number of Customers,Montly Target,Zip Code,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Calendar_Month,Calendar_year
Client Type,,,,,,,,,,,,,,,
Large Facility,1380.842105,71.578947,1003.0,1.999880e+07,143.098684,142273.609649,8.192056e+05,45595.436623,133667.763158,2.034013e+06,2.017039e+06,119287.280702,16266.447368,6.5,2014.5
Medium Facility,3940.761905,202.857143,1003.0,4.075997e+07,290.583333,437217.097817,1.552603e+06,49176.847619,398645.833333,4.822783e+06,4.698646e+06,85104.166667,33273.809524,6.5,2014.5
Private Facility,400.727273,20.454545,1003.0,5.030246e+06,35.784091,5183.715152,2.272919e+05,5522.470455,1221.590909,6.376705e+05,4.434375e+05,3664.772727,12215.909091,6.5,2014.5
Small Facility,422.514286,21.285714,1003.0,1.637759e+06,11.689286,11975.986310,9.120875e+04,0.000000,8062.500000,7.617143e+05,3.727946e+05,4223.214286,1535.714286,6.5,2014.5


In [322]:
DF.corr(numeric_only=True)[['Amount Collected']].sort_values(by=['Amount Collected'], ascending=False)

,Amount Collected
Amount Collected,1.000000
Unit Sold,0.997515
Montly Target,0.608204
Number of Customers,0.607496
Sales Contact 2,0.552112
Campaign (Flyer),0.444337
Sales Contact 3,0.357887
Calendar_year,0.286194
Sales Contact 1,0.277478
Campaign (Email),0.248235


In [323]:
## Consolidated strategy for targeting
# Calculate correlation between relevant columns
aux00 = DF[['Amount Collected',
            'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
            'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3', 'Sales Contact 4', 'Sales Contact 5']]
correlation_analysis = pd.DataFrame(aux00.corr()['Amount Collected']).reset_index()

# Change column names
correlation_analysis.columns=['Impacting Variable on Sales', 'Impact (Correlation)']
# Drop row
correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable on Sales'] != 'Amount Collected']
# Sort values
correlation_analysis = correlation_analysis.sort_values(by=['Impact (Correlation)'], ascending=False)
# Show dataframe with background colored
cm = sns.light_palette('green', as_cmap=True)
pd.io.formats.style.Styler(correlation_analysis, precision=2).background_gradient(cmap=cm)

,Impacting Variable on Sales,Impact (Correlation)
5,Sales Contact 2,0.55
2,Campaign (Flyer),0.44
6,Sales Contact 3,0.36
4,Sales Contact 1,0.28
1,Campaign (Email),0.25
7,Sales Contact 4,0.24
8,Sales Contact 5,0.10
3,Campaign (Phone),0.03


In [324]:
# Calculate correlation between relevant columns
aux00 = DF.groupby(by=["Client Type"])
aux00 = aux00[['Amount Collected',
            'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
            'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3', 'Sales Contact 4', 'Sales Contact 5']]
correlation_analysis = pd.DataFrame(aux00.corr()['Amount Collected']).reset_index()
# Change column names
correlation_analysis.columns=['Client Type', 'Impacting Variable on Sales', 'Impact (Correlation)']
# Drop row
correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable on Sales'] != 'Amount Collected']
# Sort values
correlation_analysis = correlation_analysis.sort_values(by=['Client Type' , 'Impact (Correlation)'], ascending=False)
# Show dataframe with background colored
cm = sns.light_palette('green', as_cmap=True)
pd.io.formats.style.Styler(correlation_analysis, precision=2).background_gradient(cmap=cm)

,Client Type,Impacting Variable on Sales,Impact (Correlation)
32,Small Facility,Sales Contact 2,0.22
33,Small Facility,Sales Contact 3,0.07
28,Small Facility,Campaign (Email),0.06
29,Small Facility,Campaign (Flyer),0.04
34,Small Facility,Sales Contact 4,0.02
35,Small Facility,Sales Contact 5,0.00
31,Small Facility,Sales Contact 1,-0.02
30,Small Facility,Campaign (Phone),nan
23,Private Facility,Sales Contact 2,0.57
20,Private Facility,Campaign (Flyer),0.28


## Regression Analysis (Market sales and strategies)

Previously, we saw the general linear effect (or impact) that each variable has on the "Amount Collected" value. Next, we will establish an explicit relationship (linear regression) between these variables. Thus, we will directly estimate how much "Amount Collected" will increase in response to an increase in each variable.

In [327]:
# Change some characters in columns names
DF.columns = [mystring.replace(" ", "_") for mystring in DF.columns]
DF.columns = [mystring.replace("(", "") for mystring in DF.columns]
DF.columns = [mystring.replace(")", "") for mystring in DF.columns]
DF
# Linear model across all Client Types
results = smf.ols('Amount_Collected ~ Campaign_Email + Campaign_Flyer + Campaign_Phone+\
                    Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + Sales_Contact_4 + Sales_Contact_5', data=DF).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     342.1
Date:                Wed, 18 Sep 2024   Prob (F-statistic):               0.00
Time:                        00:13:03   Log-Likelihood:                -54512.
No. Observations:                2976   AIC:                         1.090e+05
Df Residuals:                    2967   BIC:                         1.091e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.481e+06   5.12e+05     

In [328]:
# Filter the variables that have a p-value lower than 0.05
df = pd.read_html(StringIO(results.summary().tables[1].as_html()), header=0, index_col=0)[0]
df.reset_index(inplace=True)
df = df[ df['P>|t|'] < 0.05 ]
df = df[['index', 'coef']]
df

,index,coef
0,Intercept,1.481000e+06
2,Campaign_Flyer,3.337600e+00
4,Sales_Contact_1,4.236800e+00
5,Sales_Contact_2,3.638200e+00
6,Sales_Contact_3,2.343200e+00
7,Sales_Contact_4,1.094780e+01


In [329]:
# Create linear models for each Client Type
consolidated_summary = pd.DataFrame()

for client_type in DF['Client_Type'].unique().tolist():
    temp_data = DF[DF['Client_Type'] == client_type].copy()
    results = smf.ols('Amount_Collected ~ Campaign_Email + Campaign_Flyer + Campaign_Phone+\
                    Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + Sales_Contact_4 + Sales_Contact_5', data=temp_data).fit()
    df = pd.read_html(StringIO(results.summary().tables[1].as_html()), header=0, index_col=0)[0]
    df.reset_index(inplace=True)
    df = df[ df['P>|t|'] < 0.05 ]
    df = df[['index', 'coef']]
    df.columns = ['Variable', 'Coefficient (Impact)']
    df['Client Type'] = client_type
    df.sort_values('Coefficient (Impact)', ascending = False, inplace=True)
    df = df[ df['Variable']!= 'Intercept' ]
    consolidated_summary = pd.concat([consolidated_summary, df])

# Final recomendations (optimal sales)

In [331]:
consolidated_summary

,Variable,Coefficient (Impact),Client Type
2,Campaign_Flyer,4.105900,Medium Facility
5,Sales_Contact_2,3.577800,Medium Facility
4,Sales_Contact_1,3.136500,Medium Facility
6,Sales_Contact_3,2.117400,Medium Facility
4,Sales_Contact_1,11.673100,Large Facility
7,Sales_Contact_4,10.614500,Large Facility
5,Sales_Contact_2,4.003100,Large Facility
2,Campaign_Flyer,2.720400,Large Facility
6,Sales_Contact_3,2.031600,Large Facility
3,Campaign_Phone,-3.536100,Large Facility


Finally, with this model we can estimate how much each dollar of each variable will increase the "Amount Collected". This allows us to decide which variables to put more effort (money) into and how much we can expect in return, when developing a marketing campaign.

For example, for each dollar spent in "Sales_Contat_1", in a "Large Facility", we will have a return of 11.63 dollars.

One consideration is that this model does not take into account the combination of these variables could make for the final result. It only considers the individual (isolated) effects in the "Amount Collected".